## DSTA

### Chapter IV - WWW, Wiki and Online social networks.

#### This __exercise__ notebook is taken from the notebook for Ch. 4 of Caldarelli-Cheesa's textbook (CC).

Please see the [class repository]() for the datasets and the __solution notebook__.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import networkx as nx


#### Get data from The Laboratory for Web Algorithmics

#### This is the page with the datasets: http://law.di.unimi.it/datasets.php

It is possible to download a network in a WebGraph format that is a compressed binary format.

The project provides various clients to extract the network strcture, in Java, C++ and in Python, py-web-graph: http://webgraph.di.unimi.it/.

In particular we got the graph and the related urls associated to each node of the .eu domain in 2005: http://law.di.unimi.it/webdata/eu-2005/.

 We exctracted the graph in a form of an edge list and we also got the file with the list of urls in the same order of the node_id

In [ ]:
ARCSFILE = './data/eu-2005_1M.arcs'
URLSFILE = './data/eu-2005.urls'

In [ ]:
#defining the eu directed graph
eu_DG = nx.DiGraph()
#retrieve just the portion of the first 1M edges of the .eu domain
#crawled in 2005
eu_DG = nx.read_edgelist(ARCSFILE, create_using = nx.DiGraph())

#generate the dictionary of node_is -> urls
file_urls = open(URLSFILE)

count = 0

dic_nodid_urls = {}

while True:
    next_line = file_urls.readline()

    if not next_line:
        break

    next_line[:-1]
    dic_nodid_urls[str(count)] = next_line[:-1]
    count = count+1

file_urls.close()

#generate the strongly connected component
scc = [(len(c),c) for c in sorted( nx.strongly_connected_components \
                               (eu_DG), key=len, reverse=True)][0][1]

eu_DG_SCC = eu_DG.subgraph(scc)


In [ ]:
l = [e for e in eu_DG_SCC.edges]

In [ ]:
l[:5]

#### Retrieving data through the  [Twitter API](https://dev.twitter.com/docs) usign the [Twython](http://twython.readthedocs.org/en/latest/) module

This part is not in use anymore as the TwitterAPI does not generally serve data anymore: we get a `403` error.

Please proceed to the 'HITS algorithm' section below.

## Hits algorithm

##### Create a simple labeled network: the 'four triangles' network

In [ ]:
DG = nx.DiGraph()

DG.add_edges_from([('A','B'),('B','C'),('A','D'), \
                   ('D','B'),('C','D'),('C','A')])

#plot the graph
nx.draw(DG, with_labels = True)

The network has a certain symmetry: each node has in-degree of 2 and out-degree of 1 or vice versa.


#### Direct implementation of the [HITS algorithm](https://en.wikipedia.org/wiki/HITS_algorithm) by [Kleinberg](https://en.wikipedia.org/wiki/Jon_Kleinberg).

In [ ]:
def HITS_algorithm(DG, K=1000):
    ''' input: -a networkx DiGraph
               -the K maximum number of iterations

        output: two dictionaries containing the hub and authority scores, resp.
    '''

    auth={}
    hub={}


    for n in DG.nodes():
        auth[n]=1.0
        hub[n]=1.0

    for k in range(K):

        norm = 0.0

        for n in DG.nodes():

            auth[n]=0.0

            # REMINDER: a predecessor of a node n is a node m
            # such that there is a direct edge from m to n
            for p in DG.predecessors(n):
                auth[n] += hub[p]

            norm += auth[n]**2.0

        norm = norm**0.5

        for n in DG.nodes():
            auth[n] = auth[n]/norm

        norm=0.0

        for n in DG.nodes():
            hub[n] = 0.0

            for s in DG.successors(n):
                hub[n] += auth[s]

            norm += hub[n]**2.0

        norm=norm**0.5

        for n in DG.nodes():
            hub[n]=hub[n]/norm

        return auth,hub


#### Let's put HITS to test.

In [ ]:
(auth, hub) = HITS_algorithm(DG, K=100)

print (auth)
print (hub)

### Q1.  Use built in hits function to find hub and authority scores.

Can you spot the differences in result?

In [ ]:
nx.draw_networkx(DG, with_labels = True)

# your solution here.

#### Adjacency matrix representation with basic operations

We refrain from using the standard `Numpy` methods for transposing and multiplying matrices.

In [ ]:
def matrix_transpose(M):

    M_out=[]

    for c in range(len(M[0])):

        M_out.append([])

        for r in range(len(M)):
            M_out[c].append(M[r][c])

    return M_out


def matrix_multiplication(M1, M2):

    M_out=[]

    for r in range(len(M1)):

        M_out.append([])

        for j in range(len(M2[0])):
            e=0.0

            for i in range(len(M1[r])):
                e+=M1[r][i]*M2[i][j]

            M_out[r].append(e)

    return M_out


Now, let's test the home-brew functions.

In [ ]:

adjacency_matrix1=[
                  [0,1,0,1],
                  [1,0,1,1],
                  [0,1,0,0]
                  ]

adjacency_matrix2 = matrix_transpose(adjacency_matrix1)

print ("Transpose adjacency matrix:", adjacency_matrix2)

res_mul = matrix_multiplication(adjacency_matrix1, adjacency_matrix2)

print ("Matrix multiplication:", res_mul)

Differently from the `Numpy` methods, our functions work with pure lists.

In [ ]:
type(res_mul)

### The Power-iterations algorithm: a direct implementation

In [ ]:
adjacency_matrix=[
                  [0,1,0,1],
                  [1,0,1,1],
                  [0,1,0,0],
                  [1,1,0,0]
                  ]
vector=[
        [0.21],
        [0.34],
        [0.52],
        [0.49]
        ]

# For small examples, few iterations will be needed.
C = 100

In [ ]:
for i in range(C):
    res = matrix_multiplication(adjacency_matrix, vector)

    norm_sq = 0.0

    for r in res:
        norm_sq = norm_sq+r[0]*r[0]

    vector = []

    for r in res:
         vector.append([r[0]/(norm_sq**0.5)])

print ("Maximum eigenvalue (in absolute value):", norm_sq**0.5)
print ("Eigenvector for the maximum eigenvalue:", vector)


#### Putting it all together: computing HITS for the WWW strongly-connected component of the `.eu` domain

In [ ]:
# Use operator.itemgetter(1) to sort the dictionary by value
import operator

In [ ]:
# Your solution here

#Please assign your results to lists sorted_auth and sorted_hub, respectively.




In [ ]:
#top ranking auth
print ("Top 5 by auth")

for p in sorted_auth[:5]:
    print (dic_nodid_urls[p[0]], p[1])

#top ranking hub
print ("Top 5 by hub")

for p in sorted_hub[:5]:
    print (dic_nodid_urls[p[0]], p[1])

### Q2. Run the built-in `nx.hits` function; can you spot the differences in result?

In [ ]:
# Your solution here

#Please assign your results to lists sorted_auth and sorted_hub, respectively.


In [ ]:


print ("Top-5 auth nodes:")

for p in sorted_auth[:5]:
    print (dic_nodid_urls[p[0]], p[1])

print ("Top-5 hub nodes:")

for p in sorted_hub[:5]:
    print (dic_nodid_urls[p[0]], p[1])

#### Compute the PageRank

In [ ]:
def pagerank(graph, damping_factor = 0.85, max_iterations = 100, min_delta = 0.00000001):

    nodes = graph.nodes()

    graph_size = len(nodes)

    if graph_size == 0:
        return {}

    # itialize the page rank dict with 1/N for all nodes
    pagerank = dict.fromkeys(nodes, (1.0-damping_factor)*1.0/ graph_size)

    min_value = (1.0-damping_factor)/len(nodes)

    for i in range(max_iterations):
        #total difference compared to last iteraction
        diff = 0

        # computes each node PageRank based on inbound links
        for node in nodes:
            rank = min_value

            for referring_page in graph.predecessors(node):
                rank += damping_factor * pagerank[referring_page]/ \
                len(list(graph.neighbors(referring_page)))

            diff += abs(pagerank[node] - rank)

            pagerank[node] = rank

        #stop if PageRank has converged
        if diff < min_delta:
            break

    return pagerank

#### The Networkx version of [PageRank](http://networkx.github.io/documentation/latest/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)

In [ ]:
G = nx.DiGraph()

G.add_edges_from([(1, 2), (2, 3), (3, 4), (3, 1), (4, 2)])
#plot the network

nx.draw(G, with_labels = True)

#our Page Rank algorithm
res_pr=pagerank(G, max_iterations = 10000, min_delta = 0.00000001, damping_factor = 0.85)
print (res_pr)

#Networkx Pagerank function
print (nx.pagerank(G,max_iter = 10000))

#### The Twitter Mention Network

Please skip this section as we don't access Twitter/X data anymore; proceed to he `Scwiki` section below.

#### Community Detection for the `Scwiki` network

In [ ]:
SCWIKI = './data/scwiki_edgelist.dat'

TITLES = './data/scwiki_page_titles.dat'

Warning: in `.eu` there are pages in the Sardinian language (and perhaps others) which require a specific coding on your own platform.

In [ ]:
#load the directed and undirected version og the scwiki graph
scwiki_pagelinks_net_dir = nx.read_edgelist(SCWIKI, create_using = nx.DiGraph())

scwiki_pagelinks_net = nx.read_edgelist(SCWIKI)

#load the page titles
diz_titles = {}

file_titles = open(TITLES, 'r')

while True:
    next_line = file_titles.readline()

    if not next_line:
        break

    print (next_line.split()[0], next_line.split()[1])

    diz_titles[next_line.split()[0]] = next_line.split()[1]

file_titles.close()